# Tourism-Large data DeepAR

<a href="https://colab.research.google.com/github/Nixtla/hierarchicalforecast/blob/main/nbs/examples/AustralianDomesticTourism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In many cases, only the time series at the lowest level of the hierarchies (bottom time series) are available. `HierarchicalForecast` has tools to create time series for all hierarchies. In this notebook we will see how to do it.

In [1]:

# compute base forecast no coherent
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive
import pandas as pd

#obtain hierarchical reconciliation methods and evaluation
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut
from datasetsforecast.hierarchical import HierarchicalData
import numpy as np
from statsforecast.models import ETS


/home/arindam/anaconda3/envs/kdd23/lib/python3.9/site-packages/statsforecast/core.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## Aggregate bottom time series

In this example we will use the [Tourism](https://otexts.com/fpp3/tourism.html) dataset from the [Forecasting: Principles and Practice](https://otexts.com/fpp3/) book. The dataset only contains the time series at the lowest level, so we need to create the time series for all hierarchies.

In [2]:
# Load TourismSmall dataset
Y_df, S, tags = HierarchicalData.load('./data', 'TourismLarge')
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

In [3]:
Y_df

,unique_id,ds,y
0,TotalAll,1998-01-01,45151.071280
1,TotalAll,1998-02-01,17294.699551
2,TotalAll,1998-03-01,20725.114184
3,TotalAll,1998-04-01,25388.612353
4,TotalAll,1998-05-01,20330.035211
...,...,...,...
126535,GBDOth,2016-08-01,0.000000
126536,GBDOth,2016-09-01,0.000000
126537,GBDOth,2016-10-01,0.000000
126538,GBDOth,2016-11-01,0.000000


In [4]:
unq_ids = Y_df["unique_id"].unique()
num_series = len(unq_ids)
num_series

555

In [5]:
len_series = len(Y_df[Y_df["unique_id"] == unq_ids[0]])
len_series

228

In [6]:
S

,AAAHol,AAAVis,AAABus,AAAOth,AABHol,AABVis,AABBus,AABOth,ABAHol,ABAVis,...,GBBBus,GBBOth,GBCHol,GBCVis,GBCBus,GBCOth,GBDHol,GBDVis,GBDBus,GBDOth
TotalAll,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
AAll,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BAll,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CAll,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DAll,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GBCOth,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
GBDHol,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
GBDVis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
GBDBus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [7]:
tags

{'Country': array(['TotalAll'], dtype=object),
 'Country/State': array(['AAll', 'BAll', 'CAll', 'DAll', 'EAll', 'FAll', 'GAll'],
       dtype=object),
 'Country/State/Zone': array(['AAAll', 'ABAll', 'ACAll', 'ADAll', 'AEAll', 'AFAll', 'BAAll',
        'BBAll', 'BCAll', 'BDAll', 'BEAll', 'CAAll', 'CBAll', 'CCAll',
        'CDAll', 'DAAll', 'DBAll', 'DCAll', 'DDAll', 'EAAll', 'EBAll',
        'ECAll', 'FAAll', 'FBAll', 'FCAll', 'GAAll', 'GBAll'], dtype=object),
 'Country/State/Zone/Region': array(['AAAAll', 'AABAll', 'ABAAll', 'ABBAll', 'ACAAll', 'ADAAll',
        'ADBAll', 'ADCAll', 'ADDAll', 'AEAAll', 'AEBAll', 'AECAll',
        'AEDAll', 'AFAAll', 'BAAAll', 'BABAll', 'BACAll', 'BBAAll',
        'BCAAll', 'BCBAll', 'BCCAll', 'BDAAll', 'BDBAll', 'BDCAll',
        'BDDAll', 'BDEAll', 'BDFAll', 'BEAAll', 'BEBAll', 'BECAll',
        'BEDAll', 'BEEAll', 'BEFAll', 'BEGAll', 'BEHAll', 'CAAAll',
        'CABAll', 'CACAll', 'CBAAll', 'CBBAll', 'CBCAll', 'CBDAll',
        'CCAAll', 'CCBAll', 'CC

In [8]:
len(tags.keys())

8

### Split Train/Test sets

We use the final horizon as test set.

In [9]:
HORIZON = 12
FREQUENCY = "1M"

In [10]:
Y_test_df = Y_df.groupby('unique_id').tail(HORIZON)
Y_train_df = Y_df.drop(Y_test_df.index)

In [11]:
Y_test_df = Y_test_df.set_index('unique_id')
Y_train_df = Y_train_df.set_index('unique_id')

In [12]:
Y_train_df.groupby('unique_id').size()

unique_id
AAAAll      216
AAABus      216
AAAHol      216
AAAOth      216
AAAVis      216
           ... 
TotalAll    216
TotalBus    216
TotalHol    216
TotalOth    216
TotalVis    216
Length: 555, dtype: int64

## Computing base forecasts

The following cell computes the **base forecasts** for each time series in `Y_df` using the `auto_arima` and `naive` models. Observe that `Y_hat_df` contains the forecasts but they are not coherent.

In [13]:

from statsforecast import StatsForecast
from statsforecast.models import Theta

fcst = StatsForecast(df=Y_train_df, 
                     models=[Theta(season_length=12, decomposition_type="additive")], 
                     freq=FREQUENCY, n_jobs=-1)
Y_hat_df = fcst.forecast(h=HORIZON, fitted=True)
Y_fitted_df = fcst.forecast_fitted_values()

## Reconcile forecasts

The following cell makes the previous forecasts coherent using the `HierarchicalReconciliation` class. Since the hierarchy structure is not strict, we can't use methods such as `TopDown` or `MiddleOut`. In this example we use `BottomUp` and `MinTrace`.

In [14]:
from hierarchicalforecast.methods import BottomUp, MinTrace, ERM

reconcilers = [
    BottomUp(),
    MinTrace(method='mint_shrink'),
    MinTrace(method='ols'),
    ERM(method='reg')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_fitted_df, S=S, tags=tags)

The dataframe `Y_rec_df` contains the reconciled forecasts.

In [15]:
Y_rec_df

,ds,Theta,Theta/BottomUp,Theta/MinTrace_method-mint_shrink,Theta/MinTrace_method-ols,Theta/ERM_method-reg_lambda_reg-0.01
unique_id,,,,,,
TotalAll,2015-12-31,46503.015625,46356.394531,48710.496310,46477.690244,51034.863281
TotalAll,2016-01-31,21182.332031,21312.962891,21886.769627,21165.974455,5147.853516
TotalAll,2016-02-29,23809.474609,23504.314453,23764.516894,23771.039086,8443.508789
TotalAll,2016-03-31,28411.462891,28313.691406,29265.609444,28388.845188,22684.580078
TotalAll,2016-04-30,22404.734375,22022.148438,22064.697453,22361.453630,17510.763672
...,...,...,...,...,...,...
GBDOth,2016-07-31,1.128747,1.128747,1.469670,-0.469371,29.299889
GBDOth,2016-08-31,1.126853,1.126853,1.372861,1.228876,15.596016
GBDOth,2016-09-30,1.124958,1.124958,1.287499,0.634534,3.732074


## Evaluation 

The `HierarchicalForecast` package includes the `HierarchicalEvaluation` class to evaluate the different hierarchies and also is capable of compute scaled metrics compared to a benchmark model.

In [16]:
tags.keys()

dict_keys(['Country', 'Country/State', 'Country/State/Zone', 'Country/State/Zone/Region', 'Country/Purpose', 'Country/State/Purpose', 'Country/State/Zone/Purpose', 'Country/State/Zone/Region/Purpose'])

In [17]:
from hierarchicalforecast.evaluation import HierarchicalEvaluation

def rmse(y, y_hat):
    return np.mean(np.sqrt(np.mean((y-y_hat)**2, axis=1)))

def mase(y, y_hat, y_insample, seasonality=4):
    errors = np.mean(np.abs(y - y_hat), axis=1)
    scale = np.mean(np.abs(y_insample[:, seasonality:] - y_insample[:, :-seasonality]), axis=1)
    return np.mean(errors / scale)

def rmsse(y, y_hat, y_insample):
    errors = np.mean(np.square(y - y_hat), axis=1)
    scale = np.mean(np.square(y_insample[:, 1:] - y_insample[:, :-1]), axis=1)
    return np.mean(np.sqrt(errors / scale))

eval_tags = {}
for k in tags.keys():
    eval_tags[k] = tags[k]

evaluator = HierarchicalEvaluation(evaluators=[rmse, mase, rmsse])
evaluation = evaluator.evaluate(
        Y_hat_df=Y_rec_df, Y_test_df=Y_test_df,
        tags=eval_tags, Y_df=Y_train_df
)
evaluation = evaluation.drop('Overall')
# evaluation.columns = ['Base', 'BottomUp', 'MinTrace(mint_shrink)', 'MinTrace(ols)']
evaluation.columns = ['Base', 'BottomUp', 'MinTrace(ols)', 'MinTrace(mint_shrink)', 'ERM']
evaluation = evaluation.applymap('{:.4f}'.format)

/tmp/ipykernel_1224619/3617059200.py:25: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  evaluation = evaluation.drop('Overall')


### RMSE

The following table shows the performance measured using RMSE across levels for each reconciliation method.

In [18]:
score_df = evaluation.query('metric == "rmse"')
score_df

,,Base,BottomUp,MinTrace(ols),MinTrace(mint_shrink),ERM
level,metric,,,,,
Country,rmse,1563.9533,1810.7714,2065.1731,1589.9265,9678.9875
Country/State,rmse,431.9847,440.2136,490.5603,425.1730,1635.2459
Country/State/Zone,rmse,182.2783,182.0476,192.8063,180.3778,534.6341
Country/State/Zone/Region,rmse,91.9442,92.0643,95.0313,91.7447,240.1953
Country/Purpose,rmse,764.5150,737.6685,809.4236,755.6686,3296.9339
Country/State/Purpose,rmse,185.7652,182.6150,195.0607,185.0609,610.4452
Country/State/Zone/Purpose,rmse,77.5978,77.4555,79.9975,77.7628,208.0404
Country/State/Zone/Region/Purpose,rmse,39.2102,39.2102,40.1887,39.4392,94.7613


### MASE


The following table shows the performance measured using MASE across levels for each reconciliation method.

In [19]:
evaluation.query('metric == "mase"')

,,Base,BottomUp,MinTrace(ols),MinTrace(mint_shrink),ERM
level,metric,,,,,
Country,mase,0.1949,0.2122,0.2634,0.1966,1.2051
Country/State,mase,0.3418,0.3297,0.3588,0.3389,1.2262
Country/State/Zone,mase,0.4376,0.4391,0.4504,0.4412,1.3646
Country/State/Zone/Region,mase,0.5665,0.5672,0.5791,0.5741,1.4928
Country/Purpose,mase,0.4819,0.4373,0.4787,0.4684,1.8626
Country/State/Purpose,mase,0.5970,0.5905,0.6030,0.5979,2.0670
Country/State/Zone/Purpose,mase,0.7111,0.7130,0.7234,0.7195,2.0158
Country/State/Zone/Region/Purpose,mase,0.8080,0.8080,0.8232,0.8398,2.2586


### RMSSE

In [20]:
score_df = evaluation.query('metric == "rmsse"')
score_df

,,Base,BottomUp,MinTrace(ols),MinTrace(mint_shrink),ERM
level,metric,,,,,
Country,rmsse,0.1509,0.1747,0.1993,0.1534,0.9339
Country/State,rmsse,0.3845,0.3798,0.3997,0.3777,1.3432
Country/State/Zone,rmsse,0.4684,0.4668,0.4841,0.4653,1.4187
Country/State/Zone/Region,rmsse,0.5691,0.5687,0.5771,0.5692,1.4405
Country/Purpose,rmsse,0.4945,0.4581,0.4797,0.4823,1.9170
Country/State/Purpose,rmsse,0.5634,0.5540,0.5633,0.5630,1.9001
Country/State/Zone/Purpose,rmsse,0.6294,0.6295,0.6377,0.6334,1.7298
Country/State/Zone/Region/Purpose,rmsse,0.6685,0.6685,0.6765,0.6784,1.6887


In [21]:
score_df.astype(float).mean()

Base                     0.491088
BottomUp                 0.487513
MinTrace(ols)            0.502175
MinTrace(mint_shrink)    0.490337
ERM                      1.546488
dtype: float64

### Comparison fable

Observe that we can recover the results reported by the [Forecasting: Principles and Practice](https://otexts.com/fpp3/tourism.html). The original results were calculated using the R package [fable](https://github.com/tidyverts/fable).

![Fable's reconciliation results](./imgs/AustralianDomesticTourism-results-fable.png)

### References
- [Hyndman, R.J., & Athanasopoulos, G. (2021). "Forecasting: principles and practice, 3rd edition: 
Chapter 11: Forecasting hierarchical and grouped series.". OTexts: Melbourne, Australia. OTexts.com/fpp3 
Accessed on July 2022.](https://otexts.com/fpp3/hierarchical.html)
- [Rob Hyndman, Alan Lee, Earo Wang, Shanika Wickramasuriya, and Maintainer Earo Wang (2021). "hts: Hierarchical and Grouped Time Series". URL https://CRAN.R-project.org/package=hts. R package version 0.3.1.](https://cran.r-project.org/web/packages/hts/index.html)
- [Mitchell O’Hara-Wild, Rob Hyndman, Earo Wang, Gabriel Caceres, Tim-Gunnar Hensel, and Timothy Hyndman (2021). "fable: Forecasting Models for Tidy Time Series". URL https://CRAN.R-project.org/package=fable. R package version 6.0.2.](https://CRAN.R-project.org/package=fable)